<a href="https://colab.research.google.com/github/tiffanymoran/AlbumGAN/blob/main/CLIP_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-3zzjidnz
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-3zzjidnz


In [ ]:
import numpy as np
import torch
from pkg_resources import packaging
from torch.utils.data import Dataset, DataLoader
from torch import optim
import pandas as pd
import torch.nn as nn
from PIL import Image

print("Torch version:", torch.__version__)

Torch version: 1.12.0+cu113


In [ ]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
!git clone https://github.com/openai/CLIP

fatal: destination path 'CLIP' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

Restructure CSV for CLIP training

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Images/captionsfinal.csv',
                 names=["Path", "Drop", "Caption"])

In [1]:
df['Caption'] = df['Caption'].str[:255] #truncate lyrics to make sure they aren't too long

NameError: ignored

In [ ]:
df['Path'] = '/content/drive/MyDrive/ColabNotebooks/Images/' + df['Path'].astype(str) #add the path to the file name

In [ ]:
image_list = df.Drop.values.tolist() #create lists for training script
caption_list = df.Caption.values.tolist()

In [ ]:
class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title

In [ ]:
# use your own data
list_image_path = image_list 
list_txt = caption_list
dataset = image_title_dataset(list_image_path,list_txt)
train_dataloader = DataLoader(dataset,batch_size = 32) #Define your own dataloader

In [ ]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

EPOCH = 10

# add your own code to track the training progress.
for epoch in range(EPOCH):
  for batch in train_dataloader :
      optimizer.zero_grad()

      images,texts = batch 
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

In [ ]:
print(total_loss)

tensor(2.5664, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


Save Model

In [ ]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"/content/drive/MyDrive/ColabNotebooks/model_10.pt") #can update with preferred folder / file name

Reload Model

In [ ]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("/content/drive/MyDrive/ColabNotebooks/model_10.pt")

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
positional_embedding 	 torch.Size([77, 512])
text_projection 	 torch.Size([512, 512])
logit_scale 	 torch.Size([])
visual.class_embedding 	 torch.Size([768])
visual.positional_embedding 	 torch.Size([50, 768])
visual.proj 	 torch.Size([768, 512])
visual.conv1.weight 	 torch.Size([768, 3, 32, 32])
visual.ln_pre.weight 	 torch.Size([768])
visual.ln_pre.bias 	 torch.Size([768])
visual.transformer.resblocks.0.attn.in_proj_weight 	 torch.Size([2304, 768])
visual.transformer.resblocks.0.attn.in_proj_bias 	 torch.Size([2304])
visual.transformer.resblocks.0.attn.out_proj.weight 	 torch.Size([768, 768])
visual.transformer.resblocks.0.attn.out_proj.bias 	 torch.Size([768])
visual.transformer.resblocks.0.ln_1.weight 	 torch.Size([768])
visual.transformer.resblocks.0.ln_1.bias 	 torch.Size([768])
visual.transformer.resblocks.0.mlp.c_fc.weight 	 torch.Size([3072, 768])
visual.transformer.resblocks.0.mlp.c_fc.bias 	 torch.Size([3072])
visual.transformer.resblocks.0.mlp.c_proj.wei

In [ ]:
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          